# 부록 10.4.5: Haiku를 하위 에이전트로 사용하기

이 레시피에서는 Claude 3 Haiku 하위 에이전트 모델을 사용하여 Apple의 2023년 재무 실적 보고서에서 관련 정보를 추출하고, Claude 3 Sonnet을 사용하여 질문에 대한 응답을 생성하고 matplotlib를 사용하여 응답에 동반되는 그래프를 만드는 방법을 보여줍니다.

## 1단계: 환경 설정
먼저 필요한 라이브러리를 설치하고 Anthropic API 클라이언트를 설정합니다.

In [ ]:
pip install -qUr requirements.txt

In [ ]:
# Import the required libraries
import boto3
import fitz
from PIL import Image
import io
from concurrent.futures import ThreadPoolExecutor
import requests
import os

session = boto3.Session()
region = session.region_name

modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print('Using region: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

## 2단계: 문서 수집 및 질문 작성
이 예제에서는 2023 회계연도의 Apple 재무제표 전체를 사용하고 연간 순매출에 대해 질문할 것입니다.

In [ ]:
# List of Apple's earnings release PDF URLs
pdf_urls = [
    "https://www.apple.com/newsroom/pdfs/fy2023-q4/FY23_Q4_Consolidated_Financial_Statements.pdf",
    "https://www.apple.com/newsroom/pdfs/fy2023-q3/FY23_Q3_Consolidated_Financial_Statements.pdf",
    "https://www.apple.com/newsroom/pdfs/FY23_Q2_Consolidated_Financial_Statements.pdf",
    "https://www.apple.com/newsroom/pdfs/FY23_Q1_Consolidated_Financial_Statements.pdf"
]

# User's question
QUESTION = "How did Apple's net sales change quarter to quarter in the 2023 financial year and what were the key contributors to the changes?"

## 3단계: PDF 다운로드 및 이미지 변환
다음으로 수익 실적 보고서 PDF를 다운로드하고 base64 인코딩된 PNG 이미지로 변환하는 함수를 정의합니다. 이렇게 해야 하는 이유는 이러한 PDF에 전통적인 PDF 파서로 구문 분석하기 어려운 표가 많이 포함되어 있기 때문입니다. 이미지로 변환하여 Haiku에 전달하는 것이 더 쉽습니다.

```download_pdf``` 함수는 주어진 URL에서 PDF 파일을 다운로드하여 지정된 폴더에 저장합니다. ```pdf_to_pngs``` 함수는 PDF를 PNG 이미지 목록으로 변환합니다.

In [ ]:
# Function to download a PDF file from a URL and save it to a specified folder
def download_pdf(url, folder):
    response = requests.get(url)
    if response.status_code == 200:
        file_name = os.path.join(folder, url.split("/")[-1])
        with open(file_name, "wb") as file:
            file.write(response.content)
        return file_name
    else:
        print(f"Failed to download PDF from {url}")
        return None
    
# Define the function to convert a PDF to a list of base64-encoded PNG images
def pdf_to_png(pdf_path, quality=75, max_size=(1024, 1024)):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    pdf_to_png_images = []

    # Iterate through each page of the PDF
    for page_num in range(doc.page_count):
        # Load the page
        page = doc.load_page(page_num)

        # Render the page as a PNG image
        pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))

        # Convert the pixmap to a PIL Image
        image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # Resize the image if it exceeds the maximum size
        if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
            image.thumbnail(max_size, Image.Resampling.LANCZOS)

        # Convert the PIL image to bytes
        image_data = io.BytesIO()
        image.save(image_data, format='PNG', optimize=True, quality=quality)
        image_data.seek(0)
        pdf_to_png_image = image_data.getvalue()

        # Append the PNG image bytes to the list
        pdf_to_png_images.append(pdf_to_png_image)

    # Close the PDF document
    doc.close()

    return pdf_to_png_images

# Folder to save the downloaded PDFs
folder = "./images/using_sub_agents"

# Download the PDFs concurrently
with ThreadPoolExecutor() as executor:
    pdf_paths = list(executor.map(download_pdf, pdf_urls, [folder] * len(pdf_urls)))

# Remove any None values (failed downloads) from pdf_paths
pdf_paths = [path for path in pdf_paths if path is not None]

ThreadPoolExecutor를 사용하여 PDF를 동시에 다운로드하고 파일 경로를 pdf_paths에 저장합니다.

## 4단계: Sonnet을 사용하여 Haiku에 대한 특정 프롬프트 생성
Opus를 오케스트레이터로 사용하고 사용자가 제공한 질문을 기반으로 각 Haiku 하위 에이전트에 대한 특정 프롬프트를 작성하도록 합니다.

In [ ]:
def generate_haiku_prompt(question):
    prompt = f"""Based on the following question, please generate a specific prompt for an LLM sub-agent to extract relevant information from an earning's report PDF. Each sub-agent only has access to a single quarter's earnings report. Output only the prompt and nothing else.\n\nQuestion: {question}"""
    messages = [
        {
            "role": 'user',
            "content": [
                {"text": prompt }
            ]
        }
    ]

    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
    }

    response = bedrock_client.converse(**converse_api_params)

    return response['output']['message']['content'][0]['text']

haiku_prompt = generate_haiku_prompt(QUESTION)
print(haiku_prompt)

## 5단계: PDF에서 정보 추출
이제 질문을 정의하고 하위 에이전트 Haiku 모델을 사용하여 PDF에서 정보를 추출합니다. 각 모델에서 추출한 정보를 깔끔하게 정의된 XML 태그 집합으로 포맷팅합니다.

In [ ]:
def extract_info(pdf_path, haiku_prompt):
    pdf_pngs = pdf_to_png(pdf_path)

    messages = [
        {
            "role": "user",
            "content": [
                *[{"image": {"format": 'png', "source": {"bytes": pdf_png}}} for pdf_png in pdf_pngs],
                {"text": haiku_prompt}
            ]
        }
    ]

    converse_api_params = {
        "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
        "messages": messages,
    }
    response = bedrock_client.converse(**converse_api_params)

    return response['output']['message']['content'][0]['text'], pdf_path

def process_pdf(pdf_path):
    return extract_info(pdf_path, QUESTION)

# Process the PDFs concurrently with Haiku sub-agent models
with ThreadPoolExecutor() as executor:
    extracted_info_list = list(executor.map(process_pdf, pdf_paths))

extracted_info = ""
# Display the extracted information from each model call
for info in extracted_info_list:
    extracted_info += "<info quarter=\"" + info[1].split("/")[-1].split("_")[1] + "\">" + info[0] + "</info>\n"
print(extracted_info)

하위 에이전트 모델을 사용하여 PDF에서 동시에 정보를 추출하고 추출된 정보를 결합합니다. 그런 다음 질문과 추출된 정보를 포함하여 강력한 모델에 대한 메시지를 준비하고, 응답과 matplotlib 코드를 생성하도록 요청합니다.

## 6단계: Sonnet에 정보를 전달하여 응답 생성
이제 하위 에이전트를 사용하여 각 PDF에서 정보를 가져왔으므로 Opus를 호출하여 실제로 질문에 답변하고 응답에 동반되는 그래프를 만들기 위한 코드를 작성합니다.

In [ ]:
# Prepare the messages for the powerful model
messages = [
    {
        "role": "user",
        "content": [
            {"text": f"Based on the following extracted information from Apple's earnings releases, please provide a response to the question: {QUESTION}\n\nAlso, please generate Python code using the matplotlib library to accompany your response. Enclose the code within <code> tags.\n\nExtracted Information:\n{extracted_info}"}
        ]
    }
]

# Generate the matplotlib code using the powerful model
converse_api_params = {
    "modelId": "anthropic.claude-3-sonnet-20240229-v1:0",
    "messages": messages,
    "inferenceConfig": {"maxTokens": 4096},
}
response = bedrock_client.converse(**converse_api_params)

generated_response = response['output']['message']['content'][0]['text']
print("Generated Response:")
print(generated_response)

## 7단계: 응답 추출 및 Matplotlib 코드 실행
마지막으로 생성된 응답에서 matplotlib 코드를 추출하고 실행하여 수익 성장 추세를 시각화합니다.

```extract_code_and_response``` 함수를 정의하여 생성된 응답에서 matplotlib 코드와 코드가 아닌 응답을 추출합니다. 코드가 아닌 응답을 인쇄하고 matplotlib 코드가 있으면 실행합니다.

모델이 작성한 코드를 샌드박스 외부에서 ```exec```를 사용하는 것은 좋은 관행이 아니지만 이 데모를 위해 그렇게 하겠습니다 :)

In [ ]:
# Extract the matplotlib code from the response
# Function to extract the code and non-code parts from the response
def extract_code_and_response(response):
    start_tag = "<code>"
    end_tag = "</code>"
    start_index = response.find(start_tag)
    end_index = response.find(end_tag)
    if start_index != -1 and end_index != -1:
        code = response[start_index + len(start_tag):end_index].strip()
        non_code_response = response[:start_index].strip()
        return code, non_code_response
    else:
        return None, response.strip()

matplotlib_code, non_code_response = extract_code_and_response(generated_response)

print(non_code_response)
if matplotlib_code:

    # Execute the extracted matplotlib code
    exec(matplotlib_code)
else:
    print("No matplotlib code found in the response.")